Run Locally so we need to get pwd

In [2]:
import os

os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'Assignment_Statement.pdf',
 'Classification_Native_Speaker.ipynb',
 'ECE 114 Speech Accent Classification Project.ipynb',
 'extracted_features',
 'feat_comb.mat',
 'feat_vec.mat',
 'feat_vec_rnn_example.mat',
 'formant_rnn.mat',
 'labels.mat',
 'MFCC.mat',
 'Notes.txt',
 'Preliminary Results.docx',
 'README.md',
 'speech_data',
 'VoiceSauce',
 'voicesauce_paper_exp.pdf',
 'voicesause_output2featvec_example.m']

Load Data for CNN

In [5]:
import numpy as np
import scipy.io as sio
import sklearn
from sklearn.model_selection import train_test_split

# Load feature data from .mat file
X=sio.loadmat('feat_vec.mat')
X_data=X['feat_vec']

# Load labels from .mat file
y=sio.loadmat('labels.mat')
y_data=y['labels']

# Partition testing set and training / validation
X_train_val, X_test, y_train_val, y_test= train_test_split(X_data, y_data, test_size=0.15)

# Partition into training / validation set
X_train, X_val, y_train, y_val= train_test_split(X_train_val, y_train_val, test_size=0.15)

print ('Training/Valid data shape: {}'.format(X_train_val.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_val.shape))
print ('Test target shape: {}'.format(y_test.shape))

Training/Valid data shape: (144, 12)
Test data shape: (26, 12)
Training/Valid target shape: (144, 1)
Test target shape: (26, 1)


One-hot encode labels

In [6]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

y_train = onehot_encoder.fit_transform(y_train)
y_val = onehot_encoder.fit_transform(y_val)
y_test = onehot_encoder.fit_transform(y_test)

print(y_train.shape)

(122, 2)


Run Fully Connected (FeedForward) Network

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers

model = Sequential()

# Softmax layer chosen emperically
# Dropout layer serves to prevent overfitting by simply dropping some outputs
model.add(Dense(128, activation='softmax', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))         

Adam=optimizers.Nadam()

model.compile(loss='binary_crossentropy',
              optimizer=Adam,
              metrics=['accuracy'])

history=model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=100, epochs=100)

Train on 122 samples, validate on 22 samples
Epoch 1/100
122/122 [==============================] - 2s 13ms/sample - loss: 2.0075 - accuracy: 0.5123 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 2/100
122/122 [==============================] - 0s 233us/sample - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 3/100
122/122 [==============================] - 0s 246us/sample - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 4/100
122/122 [==============================] - 0s 264us/sample - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 5/100
122/122 [==============================] - 0s 272us/sample - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 6/100
122/122 [==============================] - 0s 257us/sample - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6666 - val_accuracy: 0.5000
Epoch 7/100
122/122 [==============================] - 0s 258us/sample - l

Get validation and test scores

In [8]:
score_val = model.evaluate(X_val, y_val, batch_size = 20, verbose = 0)
score_test = model.evaluate(X_test, y_test, batch_size = 20, verbose = 0)

print('Validation [loss, accuracy] is ', score_val)
print('Test [loss, accuracy] is ', score_test)

Validation [loss, accuracy] is  [7.666619300842285, 0.5]
Test [loss, accuracy] is  [7.666619410881629, 0.5]


Get data formatted for RNN

In [3]:
import numpy as np
import scipy.io as sio
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Activation
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import BatchNormalization

# Load RNN Feature data from .mat file and format it
X=sio.loadmat('feat_comb.mat')
X_data=X['feat_comb']
X_data=np.transpose(X_data, (2, 0, 1))

# Load label data
y=sio.loadmat('labels.mat')
y_data=y['labels']

# Create testing set
X_train_val, X_test, y_train_val, y_test= train_test_split(X_data, y_data, test_size=0.15)

# Split remaining data into training and validation sets
X_train, X_val, y_train, y_val= train_test_split(X_train_val, y_train_val, test_size=0.15)

# Encode and reshape the data for input into the RNN
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

y_train = onehot_encoder.fit_transform(y_train)
y_val = onehot_encoder.fit_transform(y_val)
y_test = onehot_encoder.fit_transform(y_test)

# Average validation and test accuracy for each model
avg_val = 0
avg_test = 0

# Define each dimension of our feature matrix
# Dimension 0 represents each voice clip
# Dimension 1 represents each feature for each voice clip
# Dimension 2 represents how each feature changes across time
n_samples = X_train.shape[0]
n_features = X_train.shape[1]
time_steps = X_train.shape[2]

for i in range(25):
    
    # Reassign training, validation, and testing sets for each model
    X_train_val, X_test, y_train_val, y_test= train_test_split(X_data, y_data, test_size=0.15)
    X_train, X_val, y_train, y_val= train_test_split(X_train_val, y_train_val, test_size=0.15)
    y_train = onehot_encoder.fit_transform(y_train)
    y_val = onehot_encoder.fit_transform(y_val)
    y_test = onehot_encoder.fit_transform(y_test)
    
    # Create the RNN Model (for some reason, this had to be redeclared each time or else the loss from each trial would carry over)
    # Normalization used per the internet's suggestion, something to do with preventing values from shifting around too much (covariance shift)
    # Played around with LSTM sizes as well but it didn't have too much effect 
    model = Sequential()
    model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_initializer='random_normal', input_shape=(n_features, time_steps)))
    model.add(MaxPooling1D(pool_size=(2), strides=2, padding='valid', data_format=None))
    model.add(LSTM(40, return_sequences=True))
    model.add(LSTM(30, return_sequences=False))
    
    model.add(Dense(32, kernel_regularizer=regularizers.l1_l2(l1=0,l2=0.5), use_bias='false'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Activation("sigmoid"))

    model.add(Dense(16, kernel_regularizer=regularizers.l1_l2(l1=0,l2=0.5), use_bias='false'))
    model.add(BatchNormalization())
    model.add(Activation("sigmoid"))

    model.add(Dense(4, kernel_regularizer=regularizers.l1_l2(l1=0,l2=0.5), use_bias='false'))
    model.add(BatchNormalization())
    model.add(Activation("sigmoid"))

    model.add(Dense(2, kernel_regularizer=regularizers.l1_l2(l1=0,l2=0.5), use_bias='false'))
    model.add(BatchNormalization())
    model.add(Activation("sigmoid"))

    model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])
    print("Trial", i, "with Sigmoid Only and MFCC Features")
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=100, epochs=15, verbose=0)

    score_val = model.evaluate(X_val, y_val, batch_size=20, verbose=0)
    score_test = model.evaluate(X_test, y_test, batch_size=20, verbose=0)

    avg_val += score_val[1]/25
    avg_test += score_test[1]/25
    
    print('Validation [loss, accuracy] is ', score_val)
    print('Test [loss, accuracy] is ', score_test)
    print("\n")

print('Avg Validation accuracy is ', avg_val)
print('Avg Test accuracy is ', avg_test, "\n")

Trial 0 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [2.3715668591586025, 0.54545456]
Test [loss, accuracy] is  [2.375562649506789, 0.8076923]


Trial 1 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [2.4657118537209253, 0.59090906]
Test [loss, accuracy] is  [2.465595318720891, 0.61538464]


Trial 2 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [1.8445332808928057, 0.59090906]
Test [loss, accuracy] is  [1.8450641081883357, 0.5769231]


Trial 3 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [2.2417446916753594, 0.59090906]
Test [loss, accuracy] is  [2.2425534358391395, 0.53846157]


Trial 4 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [2.457397872751409, 0.6818182]
Test [loss, accuracy] is  [2.477993139853844, 0.46153846]


Trial 5 with Sigmoid Only and MFCC Features
Validation [loss, accuracy] is  [1.909371170130643, 0.6818182]
Test [loss, accuracy] is  [1.8992739273951604, 0.6